In [1088]:
#Import Libraries
import math
import nltk
from nltk import sent_tokenize, word_tokenize, PorterStemmer
from nltk.corpus import stopwords
import os
import re
import operator
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize,word_tokenize
#nltk.download ('all')
Stopwords = set(stopwords.words('english'))
wordlemmatizer = WordNetLemmatizer()

In [1089]:
#input article 1 (newspaper article)
#f = open ("Flood_Malaysia.txt", "r")
#text = (f.read())
#print (text)

In [1090]:
#input article 2 (research paper)
#f = open ("Covid_article.txt", "r")
#text = (f.read())
#print (text)

In [1091]:
#input article 3 (web article)
f = open ("Economy_Malaysia.txt", "r") #
text = (f.read())
#print (text)

In [1092]:
#Text preprocessing
tokenized_sentence = sent_tokenize(text)
text = re.sub(r'\d+', '', text)
tokenized_words_with_stopwords = word_tokenize(text)
tokenized_words = [word for word in tokenized_words_with_stopwords if word not in Stopwords]
tokenized_words = [word for word in tokenized_words if len(word) > 1]
tokenized_words = [word.lower() for word in tokenized_words]

In [1093]:
#Removal of special character "^a-zA-Z0-9\s"
def remove_special_characters(text):
    regex = r'[^a-zA-Z0-9\s]'
    text = re.sub(regex,'',text)
    return text


In [1094]:
# Sentence Tokenize
'''
    We already have a sentence tokenizer, so we just need 
    to run the sent_tokenize() method to create the array of sentences.
'''
sentences = sent_tokenize(text)
total_documents = len(sentences)
print("Sentences: ")
#print(sentences)

Sentences: 


In [1095]:
# Function to pos tag all the words in the text and returns only the nouns and verbs from the text
def pos_tagging(text):
    pos_tag = nltk.pos_tag(text.split())
    pos_tagged_noun_verb = []
    for word,tag in pos_tag:
        if tag == "NN" or tag == "NNP" or tag == "NNS" or tag == "VB" or tag == "VBD" or tag == "VBG" or tag == "VBN" or tag == "VBP" or tag == "VBZ":
            pos_tagged_noun_verb.append(word)
    return pos_tagged_noun_verb

In [1096]:
#Create frequency table
def _create_frequency_table(text_string) -> dict:
    
    """
    we create a dictionary for the word frequency table.
    For this, we only used the words that are not part of the stopWords array.
    Removing stop words and making frequency table
    Stemmer - an algorithm to bring words to its root word.
    """
    stopWords = set(stopwords.words("english"))
    words = word_tokenize(text_string)
    ps = PorterStemmer()

    freqTable = dict()
    for word in words:
        word = ps.stem(word)
        if word in stopWords:
            continue
        if word in freqTable:
            freqTable[word] += 1
        else:
            freqTable[word] = 1

    return freqTable

In [1097]:
#Create frequency matrix of the words in each sentence
def _create_frequency_matrix(sentences):
    frequency_matrix = {}
    stopWords = set(stopwords.words("english"))
    ps = PorterStemmer()

    for sent in sentences:
        freq_table = {}
        words = word_tokenize(sent)
        for word in words:
            word = word.lower()
            word = ps.stem(word)
            if word in stopWords:
                continue

            if word in freq_table:
                freq_table[word] += 1
            else:
                freq_table[word] = 1

        frequency_matrix[sent[:15]] = freq_table

    return frequency_matrix

In [1098]:
freq_matrix = _create_frequency_matrix(sentences)
print("Frequency matrix: ")
#print(freq_matrix)

Frequency matrix: 


In [1099]:
#Create term- frequency matrix by calculating tf for each word
def _create_tf_matrix(freq_matrix):
    tf_matrix = {}

    for sent, f_table in freq_matrix.items():
        tf_table = {}

        count_words_in_sentence = len(f_table)
        for word, count in f_table.items():
            tf_table[word] = count / count_words_in_sentence

        tf_matrix[sent] = tf_table

    return tf_matrix

In [1100]:
#Calculate TermFrequency and generate a matrix
#Term frequency (TF) is how often a word appears in a document, divided by how many words are there in a document
tf_matrix = _create_tf_matrix(freq_matrix)
print("tf matrix: ")
#print(tf_matrix)

tf matrix: 


In [1101]:
#Function for per word document to score calculation by idf matrix
def _create_documents_per_words(freq_matrix):
    word_per_doc_table = {}

    for sent, f_table in freq_matrix.items():
        for word, count in f_table.items():
            if word in word_per_doc_table:
                word_per_doc_table[word] += 1
            else:
                word_per_doc_table[word] = 1

    return word_per_doc_table

In [1102]:
#creating table for per words apeears in documents
count_doc_per_words = _create_documents_per_words(freq_matrix)
print("Count doc per word matrix: ")
#print(count_doc_per_words)

Count doc per word matrix: 


In [1103]:
#Inverse document frequency matrix calculation each word in the sentence
def _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents):
    idf_matrix = {}

    for sent, f_table in freq_matrix.items():
        idf_table = {}

        for word in f_table.keys():
            idf_table[word] = math.log10(total_documents / float(count_doc_per_words[word]))

        idf_matrix[sent] = idf_table

    return idf_matrix

In [1104]:
#Calculate IDF and generate a matrix
#Inverse document frequency (IDF) is how unique or rare a word is.
    
idf_matrix = _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents)
print("idf matrix: ")
#print(idf_matrix)

idf matrix: 


In [1105]:
# A function for term frequency-inverse document frequency matrix by calculating tf-idf
def _create_tf_idf_matrix(tf_matrix, idf_matrix):
    tf_idf_matrix = {}

    for (sent1, f_table1), (sent2, f_table2) in zip(tf_matrix.items(), idf_matrix.items()):

        tf_idf_table = {}

        for (word1, value1), (word2, value2) in zip(f_table1.items(),
                                                    f_table2.items()):  # here, keys are the same in both the table
            tf_idf_table[word1] = float(value1 * value2)

        tf_idf_matrix[sent1] = tf_idf_table

    return tf_idf_matrix

In [1106]:
# Calculate TF-IDF and generate a matrix
tf_idf_matrix = _create_tf_idf_matrix(tf_matrix, idf_matrix)
print("tf-idf matrix: ")
#print(tf_idf_matrix)

tf-idf matrix: 


In [1107]:
#Scoring the sentecne with distinct algorithm, here we have used tf-idf score
def _score_sentences(tf_idf_matrix) -> dict:
    """
    score a sentence by its word's TF
    Basic algorithm: adding the TF frequency of every non-stop word 
    in a sentence divided by total no of words in a sentence.
    """

    sentenceValue = {}

    for sent, f_table in tf_idf_matrix.items():
        total_score_per_sentence = 0

        count_words_in_sentence = len(f_table)
        for word, score in f_table.items():
            total_score_per_sentence += score

        sentenceValue[sent] = total_score_per_sentence / count_words_in_sentence

    return sentenceValue

In [1108]:
 #score the sentences
sentence_scores = _score_sentences(tf_idf_matrix)
print("sentence scores: ")
#print(sentence_scores)

sentence scores: 


In [1109]:
#Find threshold by calculating the average sentence score.
def _find_average_score(sentenceValue) -> int:
    """
    Find the average score from the sentence value dictionary
    """
    sumValues = 0
    for entry in sentenceValue:
        sumValues += sentenceValue[entry]

    # Average value of a sentence from original summary_text
    average = (sumValues / len(sentenceValue))

    return average

In [1110]:
#Find the threshold
threshold = _find_average_score(sentence_scores)
print("Threshold: ")
print(threshold)


Threshold: 
0.06860135204014838


In [1111]:
#Generate the final summary of the praghraph

def _generate_summary(sentences, sentenceValue, threshold):
    sentence_count = 0
    summary = ''

    for sentence in sentences:
        if sentence[:15] in sentenceValue and sentenceValue[sentence[:15]] >= (threshold):
            summary += " " + sentence
            sentence_count += 1

    return summary

In [1112]:

def run_summarization(text):

    #Generate the summary
    summary = _generate_summary(sentences, sentence_scores, 1.1 * threshold)
    return summary


if __name__ == '__main__':
    result = run_summarization(text)
    print("Summary:")
    print(result)
    outF = open('SummaryEconomy_Malaysia.txt',"w")
    outF.write(result)

Summary:
 More significantly, at least on the part of those being relocated, it provokes social disruption and upheaval when people are bundled into an alien environment. Examines the effectiveness of government‐run permanent relocation schemes as a response to flood hazards in Malaysia. In Malaysia, the DID is the recognised flood management authority in the country. The DID, however, is a department dominated by engineers whose traditional training is to control floods. In reality, however, it coordinates operations at the national level and overlooks operations at the state level. Much of the operations in each state are left to be run by the respective state authorities. It is also more of a welfare body than it is a flood management organisation. The DRPC coordinates all relief operations from the Malaysian Control Centre in Kuala Lumpur.
